In [1]:
%matplotlib notebook  

In [2]:
!nvidia-smi

Wed Jul 28 11:08:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-PCIE-40GB      Off  | 00000000:01:00.0 Off |                    0 |
| N/A   33C    P0    36W / 250W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  A100-PCIE-40GB      Off  | 00000000:25:00.0 Off |                    0 |
| N/A   

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import time
import torch
import pandas as pd

# Python 3 standard library
from pathlib import Path

from model.collectdata_kde_Ellipsoids import collect_t2kde_data

##  It takes 9 input features (poca centers + (A,B,C,D,E,F)) . 
from model.models_kde import TracksToKDE_Ellipsoids_DDplus as Model

# kde_loss_Ba is a Chi2 loss (go and look at it!)
from model.kde_loss_Ba import Loss

from model.training_kde import trainNet, select_gpu, Results
from model.plots import dual_train_plots, dual_train_plots_tracks_to_KDE, replace_in_ax

In [4]:
n_epochs = 250
# Size of batches
batch_size = 64 ## batch_size = 24 ---> 4763MiB / 12066MiB on Titan V
# How fast to learn
learning_rate = 1e-5
architectures = [[5, 7, 10, 25, 50, 75, 100, 150, 250, 350, 500]]
latentChannels = 4
model_names = ['5to500_nodes_it29pretrain']

previous = ['jgo_files/28July2021_DDplus_loss_Ba_iter29_floatAll_SetVar_5to500_nodes_unstick_attpt2_50_epochs_1e-05/28July2021_DDplus_loss_Ba_iter29_floatAll_SetVar_5to500_nodes_unstick_attpt2_50_epochs_1e-05_final.pyt']

# architectures = [[5]*11,
#                  [10]*11,
#                  [15]*11
#                  [20]*11
#                  [25]*11
#                  [30]*11
#                  [50]*11
#                  [5, 6, 8, 9, 11, 12, 14, 15, 17, 18, 20],
#                  [5, 5, 5, 6, 7, 8, 10, 12, 14, 17, 20],
#                  [5, 6, 7, 8, 9, 11, 12, 14, 16, 17, 20]
#                  [5, 9, 14, 18, 23, 27, 32, 36, 41, 45, 50],
#                  [5, 7, 10, 25, 50, 75, 100, 150, 250, 350, 500]]
# model_names = ['5_nodes',
#                '10_nodes',
#                '15_nodes',
#                '20_nodes',
#                '25_nodes',
#                '30_nodes',
#                '50_nodes',
#                '5to20_lin_inc_nodes',
#                '5to20_quadly_inc_nodes',
#                '5to20_exp_inc_nodes',
#                '5to50_nodes'
#                '5to500_exp_inc_nodes']

device = select_gpu(0)
##device = "cpu"

1 available GPUs (initially using device 0):
  0 A100-PCIE-40GB


In [5]:
# Data Import. If we are using a larger dataset (240K events, with the datasets above, and 11 GB  of GPU memory),
# the dataset will overflow the GPU memory; device=device will allow the data to move back
# and forth between the CPU and GPU memory. While this allows use of a larger dataset, it slows
# down performance by about 10%.  So comment out when not needed.

## Training dataset. You can put as many files here as desired.

#train_loader = collect_t2kde_data('/share/lazy/pv-finder/20k_evts_for_KDE_learning_200716.h5',
train_loader = collect_t2kde_data('/share/lazy/will/data/June30_2020_80k_1.h5', 
                                  '/share/lazy/will/data/June30_2020_80k_2.h5',
                                    '/share/lazy/will/data/June30_2020_80k_3.h5',
                                    '/share/lazy/will/data/June30_2020_80k_4.h5',
# #                                       '/share/lazy/will/data/June30_2020_80k_5.h5',
#                                       #'/share/lazy/will/data/June30_2020_80k_6.h5',
#                                       #'/share/lazy/will/data/June30_2020_80k_7.h5',
#     train_loader = collect_t2kde_data('dataAA/20K_POCA_kernel_evts_200926.h5',
                             batch_size=batch_size,
## if we are using a larger dataset (240K events, with the datasets above, and 11 GB  of GPU memory),
## the dataset will overflow the GPU memory; device=device will allow the data to move back
## and forth between the CPU and GPU memory. While this allows use of a larger dataset, it slows
## down performance by about 10%.  So comment out when not needed.
#                               device=device,
                            slice = slice(None,10000)
                           )

# Validation dataset. You can slice to reduce the size.
## mds no separate validation set yet,

## For iter12, change slice(18000,None) to slice(10000,None)
## First, we'll see if this changes the validation cost significantly
##  Second, we will see if this reduces the validation cost fluctuations
val_loader = collect_t2kde_data('dataAA/20K_POCA_kernel_evts_200926.h5',
                            batch_size=batch_size,
#                                 device=device,
                             slice = slice(10000,None)
                           )

Loading data...


/data/home/jgocain/.local/lib/python3.8/site-packages/awkward0/persist.py:509: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  schema = schema.tostring()


pocaMx.shape =  (80000,)
nEvts =  80000
len(pocaMx[0]) =  199
len(pocaMx[1]) =  25
len(pocaMx[2]) =  369
len(pocaMx[3]) =  143
len(pocaMx[4]) =  160
majorAxis.shape =  (80000, 3)
minorAxis_1.shape =  (80000, 3)
minorAxis_2.shape =  (80000, 3)
have entered six_ellipsoid_parameters
  
 
  nEvts =  80000
 iEvt, nTrks =  0 199
 iEvt, nTrks =  1 25
 iEvt, nTrks =  2 369
 iEvt, nTrks =  3 143
 iEvt, nTrks =  4 160
 iEvt, nTrks =  5 260
 iEvt, nTrks =  6 237
 iEvt, nTrks =  7 327
 iEvt, nTrks =  8 178
 iEvt, nTrks =  9 106
A.shape =  (80000,)
majorAxis[iTrk][0][0] =  0.00045611936
majorAxis[iTrk][1][0] =  -4.8292455e-05
majorAxis[iTrk][2][0] =  0.090019904
minorAxis_1[iTrk][0][0] =  -1.8602173
minorAxis_1[iTrk][1][0] =  -17.569641
minorAxis_1[iTrk][2][0] =  4.7891795e-08
minorAxis_2[iTrk][0][0] =  -17.569414
minorAxis_2[iTrk][1][0] =  1.8601931
minorAxis_2[iTrk][2][0] =  0.0900199
  
majorAxis[iTrk][0][0] =  0.002360258
majorAxis[iTrk][1][0] =  -0.007426616
majorAxis[iTrk][2][0] =  0.3710108


majorAxis.shape =  (80000, 3)
minorAxis_1.shape =  (80000, 3)
minorAxis_2.shape =  (80000, 3)
have entered six_ellipsoid_parameters
  
 
  nEvts =  80000
 iEvt, nTrks =  0 196
 iEvt, nTrks =  1 35
 iEvt, nTrks =  2 265
 iEvt, nTrks =  3 128
 iEvt, nTrks =  4 124
 iEvt, nTrks =  5 122
 iEvt, nTrks =  6 300
 iEvt, nTrks =  7 179
 iEvt, nTrks =  8 243
 iEvt, nTrks =  9 112
A.shape =  (80000,)
majorAxis[iTrk][0][0] =  -2.440992e-05
majorAxis[iTrk][1][0] =  -0.00038087237
majorAxis[iTrk][2][0] =  0.08211532
minorAxis_1[iTrk][0][0] =  -17.631672
minorAxis_1[iTrk][1][0] =  1.130005
minorAxis_1[iTrk][2][0] =  4.7444193e-08
minorAxis_2[iTrk][0][0] =  1.1299927
minorAxis_2[iTrk][1][0] =  17.631481
minorAxis_2[iTrk][2][0] =  0.08211532
  
majorAxis[iTrk][0][0] =  -0.02277395
majorAxis[iTrk][1][0] =  0.0014473597
majorAxis[iTrk][2][0] =  0.6347583
minorAxis_1[iTrk][0][0] =  -1.1205891
minorAxis_1[iTrk][1][0] =  -17.632273
minorAxis_1[iTrk][2][0] =  2.1543272e-09
minorAxis_2[iTrk][0][0] =  -17.6208

Loading data...
pocaMx.shape =  (20000,)
nEvts =  20000
len(pocaMx[0]) =  211
len(pocaMx[1]) =  21
len(pocaMx[2]) =  20
len(pocaMx[3]) =  198
len(pocaMx[4]) =  233
majorAxis.shape =  (20000, 3)
minorAxis_1.shape =  (20000, 3)
minorAxis_2.shape =  (20000, 3)
have entered six_ellipsoid_parameters
  
 
  nEvts =  20000
 iEvt, nTrks =  0 211
 iEvt, nTrks =  1 21
 iEvt, nTrks =  2 20
 iEvt, nTrks =  3 198
 iEvt, nTrks =  4 233
 iEvt, nTrks =  5 85
 iEvt, nTrks =  6 223
 iEvt, nTrks =  7 425
 iEvt, nTrks =  8 252
 iEvt, nTrks =  9 169
A.shape =  (20000,)
majorAxis[iTrk][0][0] =  -0.00023452607
majorAxis[iTrk][1][0] =  -0.00047206535
majorAxis[iTrk][2][0] =  0.096502915
minorAxis_1[iTrk][0][0] =  -15.822749
minorAxis_1[iTrk][1][0] =  7.8608756
minorAxis_1[iTrk][2][0] =  -2.6228399e-08
minorAxis_2[iTrk][0][0] =  7.860759
minorAxis_2[iTrk][1][0] =  15.822513
minorAxis_2[iTrk][2][0] =  0.096502915
  
majorAxis[iTrk][0][0] =  -0.37655562
majorAxis[iTrk][1][0] =  0.2768704
majorAxis[iTrk][2][0] = 

In [ ]:
destinations = []

for i in range(len(model_names)):
    if previous[i] != 'None':
        folder = time.strftime('%d%B%Y') + '_DDplus_loss_Ba_iter29_floatAll_SetVar_' + model_names[i] + '_' + str(n_epochs) + '_epochs_' + str(learning_rate)
    else:
        folder = time.strftime('%d%B%Y') + '_DDplus_loss_Ba_iter29_floatAll_SetVar_' + model_names[i] + '_nopretrain_' + str(n_epochs) + '_epochs_' + str(learning_rate)
    name   = folder

    # Make an output folder named "name" (change if you want)

    ## Special instructions for those working on goofy at UC
    ## Please be very careful to make sure that your folder
    ## does not live in a subdirectory of your home directory
    ## this disk has very little capacity. Instead, use 
    ## a subdirectory in /share/lazy with a symbolic link to
    ## it in this (the notebooks) subdirectory
    folder = 'jgo_files/' + folder
    output = Path(folder)
    output.mkdir(exist_ok=True)
    
    #Data Frame for storing learning results
    results = pd.DataFrame([], columns=Results._fields)
    
    #Building the Model per settings written in 'architectures' array
    nOut1, nOut2, nOut3, nOut4, nOut5, nOut6, nOut7, nOut8, nOut9, nOut10, nOut11 = architectures[i]
    model = Model(nOut1,nOut2,nOut3,nOut4,nOut5,nOut6,nOut7,nOut8,nOut9,nOut10,nOut11)
    
    #Defining Loss Function
    loss = Loss(epsilon=3e-6)
    
    ct = 0
    freezelimit = 11
    for child in model.children():
      print('ct, child = ',ct, "  ", child)
      if ct < freezelimit:
        print("     About to set param.requires_grad=False for ct = ", ct, "params")
        for param in child.parameters():
            param.requires_grad = False 
      ct += 1
    
    model_dict = model.state_dict()
    #Transfering Model to GPU (or CPU, if chosen)
    model = model.to(device)
    
    updated_dict = model_dict
    ##print("updated_dict = ",updated_dict)
    ## when starting "ab initio", reduce biases as the bias gets summed for each track
    ## contributing to the predicted KDE
    updated_dict["layer1.bias"] = 0.005*model_dict["layer1.bias"]
    updated_dict["layer2.bias"] = 0.005*model_dict["layer2.bias"]
    updated_dict["layer3.bias"] = 0.005*model_dict["layer3.bias"]
    updated_dict["layer4.bias"] = 0.005*model_dict["layer4.bias"]
    updated_dict["layer5.bias"] = 0.005*model_dict["layer5.bias"]
    updated_dict["layer6.bias"] = 0.005*model_dict["layer6.bias"]
    updated_dict["layer7.bias"] = 0.005*model_dict["layer7.bias"]
    updated_dict["layer8.bias"] = 0.005*model_dict["layer8.bias"]
    updated_dict["layer9.bias"] = 0.005*model_dict["layer9.bias"]
    updated_dict["layer10.bias"] = 0.005*model_dict["layer10.bias"]
    updated_dict["layer11.bias"] = 0.005*model_dict["layer11.bias"]
    model.load_state_dict(updated_dict,strict=False)

    model_dict = model.state_dict()
    
    if previous[i] != 'None':
        pretrained_dict = torch.load(previous[i])
        print(" ")
        print("  for pretrained_dict")
        index = 0
        for k,v in pretrained_dict.items():
            print("index, k =  ",index,"  ",k)
            index = index+1
        ##print("model_dict instantiated")
        # 1. filter out unnecessary keys
        pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
        print("pretrained_dict iterated")
        # 2. overwrite entries in the existing state dict
        model_dict.update(pretrained_dict) 
        ##
        #   when starting from a model with a fully connected last layer rather than a convolutional layer
        # 3. load the new state dict
        #   need to use strict=False as the two models state model attributes do not agree exactly
        #   see https://pytorch.org/docs/master/_modules/torch/nn/modules/module.html#Module.load_state_dict

        model.load_state_dict(pretrained_dict,strict=False)
    
    #Defining Optimizer (Reminder to ask about that)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    # Data Import. If we are using a larger dataset (240K events, with the datasets above, and 11 GB  of GPU memory),
    # the dataset will overflow the GPU memory; device=device will allow the data to move back
    # and forth between the CPU and GPU memory. While this allows use of a larger dataset, it slows
    # down performance by about 10%.  So comment out when not needed.

    
    for result in trainNet(model, optimizer, loss, train_loader, val_loader, n_epochs, epoch_start=len(results), notebook=True):
        
        #Record Epoch Results
        results = results.append(pd.Series(result._asdict()), ignore_index=True)

        # Save each model state dictionary
        torch.save(model.state_dict(), output / f'{name}_{result.epoch}.pyt')
    
    #Save final point in model
    torch.save(model.state_dict(), output / f'{name}_final.pyt')
    results.to_hdf(output / f'{name}_stats.hdf5', 'results')
    destinations.append(output / f'{name}_stats.hdf5')
    readme = open(output / f'{name}_README.txt', 'w')
    readme.write('Model Run on ' + time.strftime('%d%B%Y') + ' \n')
    readme.write('\n')
    readme.write('Model Architecture:' + ' \n')
    childindex = 0
    for child in model.children():
        readme.write(str(childindex) + '    ' + str(child) + ' \n')
        childindex = childindex+1
    readme.write('\n')
    readme.write('First ' + str(freezelimit - 1) + ' Layers Frozen' + ' \n')
    readme.write('\n')
    readme.write('Pre-training Used: ' + previous[i] + ' \n')
    readme.write('\n')
    readme.write('Training Data Used:' + ' \n')
    for trainfile in ['/share/lazy/will/data/June30_2020_80k_1.h5', 
              '/share/lazy/will/data/June30_2020_80k_2.h5',
              '/share/lazy/will/data/June30_2020_80k_3.h5',
              '/share/lazy/will/data/June30_2020_80k_4.h5']:
        readme.write(trainfile + ' \n')
    readme.write('Training Slice Used: ' + str((None, 100)) + ' \n')
    readme.write('\n')
    readme.write('Validation Data Used:' + ' \n')
    for testfile in ['dataAA/20K_POCA_kernel_evts_200926.h5']:
        readme.write(testfile + '\n')
    readme.write('Validation Slice Used: ' + str((10000, None)) + ' \n')
    readme.write('\n')
    readme.write('Batch Size:' + str(batch_size) + ' \n')
    readme.write('\n')
    readme.write('Loss Function: Ba' + ' \n')
    readme.write('\n')
    readme.write('Optimizer Used: Adam' + ' \n')
    readme.write('\n')
    readme.write('Latent Channels: ' + str(latentChannels) + ' \n')
    readme.close()
    
    
    #

ct, child =  0    Linear(in_features=9, out_features=5, bias=True)
     About to set param.requires_grad=False for ct =  0 params
ct, child =  1    Linear(in_features=5, out_features=7, bias=True)
     About to set param.requires_grad=False for ct =  1 params
ct, child =  2    Linear(in_features=7, out_features=10, bias=True)
     About to set param.requires_grad=False for ct =  2 params
ct, child =  3    Linear(in_features=10, out_features=25, bias=True)
     About to set param.requires_grad=False for ct =  3 params
ct, child =  4    Linear(in_features=25, out_features=50, bias=True)
     About to set param.requires_grad=False for ct =  4 params
ct, child =  5    Linear(in_features=50, out_features=75, bias=True)
     About to set param.requires_grad=False for ct =  5 params
ct, child =  6    Linear(in_features=75, out_features=100, bias=True)
     About to set param.requires_grad=False for ct =  6 params
ct, child =  7    Linear(in_features=100, out_features=150, bias=True)
     Abou

Epochs:   0%|                                                           | 0/250 [00:00<?, ?it/s, train=start, …

Number of batches: train = 157, val = 157


Training:   0%|                                                                    | 0/157 [00:00<?, ?it/s, tr…

Epoch 0: train=6.03269, val=6.77757, took 24.744 s


...

Training:   0%|                                                                    | 0/157 [00:00<?, ?it/s, tr…

Epoch 18: train=5.9579, val=6.79035, took 19.856 s


Training:   0%|                                                                    | 0/157 [00:00<?, ?it/s, tr…

Epoch 19: train=5.94481, val=6.79129, took 20.124 s


Training:   0%|                                                                    | 0/157 [00:00<?, ?it/s, tr…

In [ ]:
#Print File destinations
stringer = ''
for file in destinations:
    stringer = stringer + '\'' + name + '\', '
print(stringer)

In [7]:
quit()


In [ ]:
for child in model.children():
      print(ct, "  ", child)
      ct += 1